In [1]:
import numpy as np
import pandas as pd
pd.set_option("display.max_rows", None)

# Categorical Variables

# Numerical Variables

In [7]:
usaid = pd.read_csv("USAID_GHSC-PSM_Health_Commodity_Delivery_Dataset_20240105.csv", header= 0, low_memory= False)
usaid = usaid[["Estimated Lead Time in Days", "Illustrative Price", "Order Cycle Time", 
               "Average Days Late", "Ordered Quantity", "Shipped Quantity", "Base Unit Multiplier"]]
usaid.dropna(inplace= True)

# usaid["Ordered Quantity Total"] = usaid["Ordered Quantity"] * usaid["Base Unit Multiplier"]
usaid["Shipped Quantity Total"] = usaid["Shipped Quantity"] * usaid["Base Unit Multiplier"]
usaid.drop(["Ordered Quantity", "Base Unit Multiplier"], axis= 1, inplace= True)

usaid.head(10)
usaid.to_csv("usaid.csv")